# Quick setup for pySpark and GraphFrame

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz

In [ ]:
!tar xf /content/spark-3.0.3-bin-hadoop2.7.tgz

In [ ]:
!wget -q https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.0-s_2.12/graphframes-0.8.2-spark3.0-s_2.12.jar

In [ ]:
## install graphframe library on Colab
!mv /content/graphframes-0.8.2-spark3.0-s_2.12.jar /content/spark-3.0.3-bin-hadoop2.7/jars/graphframes-0.8.2-spark3.0-s_2.12.jar

In [ ]:
!pip -q install findspark pyspark graphframes

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

In [ ]:
import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
# confirm the spark installation
spark

In [ ]:
os.environ["HADOOP_HOME"] = os.environ["SPARK_HOME"]

os.environ["PYSPARK_DRIVER_PYTHON"] = "jupyter"
os.environ["PYSPARK_DRIVER_PYTHON_OPTS"] = "notebook"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages graphframes:graphframes:0.8.2-spark3.0-s_2.12 pyspark-shell"

In [ ]:
from graphframes import *

In [ ]:
def init_spark(app_name="HelloWorldApp", execution_mode="local[*]"):
  spark = SparkSession.builder.master(execution_mode).appName(app_name).getOrCreate()
  sc = spark.sparkContext
  return spark, sc

# Importing the Data into Apache Spark

In [ ]:
!mkdir data
!wget -q https://github.com/chunsejin/KGProjects_DAU/raw/master/data/airports.csv
!wget -q https://github.com/chunsejin/KGProjects_DAU/raw/master/data/188591317_T_ONTIME.csv.gz
!unzip 188591317_T_ONTIME.csv.gz
!mv airports.csv data
!mv 188591317_T_ONTIME.csv data

In [ ]:
from graphframes import *
from pyspark import SparkContext

In [ ]:
nodes = spark.read.csv("data/airports.csv", header=False)
cleaned_nodes = (nodes.select("_c1", "_c3", "_c4", "_c6", "_c7")
                 .filter("_c3 = 'United States'")
                 .withColumnRenamed("_c1", "name")
                 .withColumnRenamed("_c4", "id")
                 .withColumnRenamed("_c6", "latitude")
                 .withColumnRenamed("_c7", "longitude")
                 .drop("_c3"))
cleaned_nodes = cleaned_nodes[cleaned_nodes["id"] != "\\N"]

In [ ]:
relationships = spark.read.csv("data/188591317_T_ONTIME.csv", header=True)
cleaned_relationships = (relationships
                         .select("ORIGIN", "DEST", "FL_DATE", "DEP_DELAY",
                                 "ARR_DELAY", "DISTANCE", "TAIL_NUM", "FL_NUM",
                                 "CRS_DEP_TIME", "CRS_ARR_TIME",
                                 "UNIQUE_CARRIER")
                         .withColumnRenamed("ORIGIN", "src")
                         .withColumnRenamed("DEST", "dst")
                         .withColumnRenamed("DEP_DELAY", "deptDelay")
                         .withColumnRenamed("ARR_DELAY", "arrDelay")
                         .withColumnRenamed("TAIL_NUM", "tailNumber")
                         .withColumnRenamed("FL_NUM", "flightNumber")
                         .withColumnRenamed("FL_DATE", "date")
                         .withColumnRenamed("CRS_DEP_TIME", "time")
                         .withColumnRenamed("CRS_ARR_TIME", "arrivalTime")
                         .withColumnRenamed("DISTANCE", "distance")
                         .withColumnRenamed("UNIQUE_CARRIER", "airline")
                         .withColumn("deptDelay",
                                     F.col("deptDelay").cast(FloatType()))
                         .withColumn("arrDelay",F.col("arrDelay").cast(FloatType()))
                         .withColumn("time", F.col("time").cast(IntegerType()))
                         .withColumn("arrivalTime",
                                     F.col("arrivalTime").cast(IntegerType()))
                         )

In [ ]:
g = GraphFrame(cleaned_nodes, cleaned_relationships)

In [ ]:
airlines_reference = (spark.read.csv("data/airlines.csv")
                      .select("_c1", "_c3")
                      .withColumnRenamed("_c1", "name")
                      .withColumnRenamed("_c3", "code"))

In [ ]:
airlines_reference = airlines_reference[airlines_reference["code"] != "null"]

# Exploratory Analysis

In [ ]:
# how many airports we have
g.vertices.count()

In [ ]:
# how many connections do we have between these airports?
g.edges.count()

# Popular Airports

In [ ]:
airports_degree = g.outDegrees.withColumnRenamed("id", "oId")

In [ ]:
full_airports_degree = (airports_degree
                        .join(g.vertices, airports_degree.oId == g.vertices.id)
                        .sort("outDegree", ascending=False)
                        .select("id", "name", "outDegree"))

In [ ]:
full_airports_degree.show(n=10, truncate=False)

In [ ]:
# a visual representation of the out‐going flights
plt.style.use('fivethirtyeight')

ax = (full_airports_degree
      .toPandas()
      .head(10)
      .plot(kind='bar', x='id', y='outDegree', legend=None))
ax.xaxis.set_label_text("")
plt.xticks(rotation=45)
plt.tight_la

Delays from ORD ( Chicago O’Hare International Airport (ORD) )

In [ ]:
delayed_flights = (g.edges
                   .filter("src = 'ORD' and deptDelay > 0")
                   .groupBy("dst")
                   .agg(F.avg("deptDelay"), F.count("deptDelay"))
                   .withColumn("averageDelay",
                               F.round(F.col("avg(deptDelay)"), 2))
                   .withColumn("numberOfDelays",
                               F.col("count(deptDelay)")))
(delayed_flights
 .join(g.vertices, delayed_flights.dst == g.vertices.id)
 .sort(F.desc("averageDelay"))
 .select("dst", "name", "averageDelay", "numberOfDelays")
 .show(n=10, truncate=False))

In [ ]:
# find the flights between those airports
from_expr = 'id = "ORD"'
to_expr = 'id = "CKB"'
ord_to_ckb = g.bfs(from_expr, to_expr)
ord_to_ckb = ord_to_ckb.select(
    F.col("e0.date"),
    F.col("e0.time"),
    F.col("e0.flightNumber"),
    F.col("e0.deptDelay"))

In [ ]:
# plotting
ax = (ord_to_ckb
      .sort("date")
      .toPandas()
      .plot(kind='bar', x='date', y='deptDelay', legend=None))
ax.xaxis.set_label_text("")
plt.tight_l

# Bad Day at SFO
Delays at an airport is known for fog-related “low ceiling” issues: San
Francisco International Airport

In [ ]:
# motifs to find the most-delayed flights going into and out of SFO on May 11, 2018.
# The motif (a)-[ab]->(b); (b)-[bc]->(c) finds flights coming into and out from the same airport.
motifs = (g.find("(a)-[ab]->(b); (b)-[bc]->(c)")
          .filter("""(b.id = 'SFO') and
(ab.date = '2018-05-11' and bc.date = '2018-05-11') and
(ab.arrDelay > 30 or bc.deptDelay > 30) and
(ab.flightNumber = bc.flightNumber) and
(ab.airline = bc.airline) and
(ab.time < bc.time)"""))

In [ ]:
result = (motifs.withColumn("delta", motifs.bc.deptDelay - motifs.ab.arrDelay)
          .select("ab", "bc", "delta")
          .sort("delta", ascending=False))

In [ ]:
result.select(
    F.col("ab.src").alias("a1"),
    F.col("ab.time").alias("a1DeptTime"),
    F.col("ab.arrDelay"),
    F.col("ab.dst").alias("a2"),
    F.col("bc.time").alias("a2DeptTime"),
    F.col("bc.deptDelay"),
    F.col("bc.dst").alias("a3"),
    F.col("ab.airline"),
    F.col("ab.flightNumber"),
    F.col("delta")
).show()

# Interconnected Airports by Airline

In [ ]:
# identify all the airlines and work out how many flights there are on each of them:
airlines = (g.edges
            .groupBy("airline")
            .agg(F.count("airline").alias("flights"))
            .sort("flights", ascending=False))

In [ ]:
full_name_airlines = (airlines_reference
                      .join(airlines, airlines.airline
                            == airlines_reference.code)
                      .select("code", "name", "flights"))

In [ ]:
# Bar chart
ax = (full_name_airlines.toPandas()
      .plot(kind='bar', x='name', y='flights', legend=None))
ax.xaxis.set_label_text("")
plt.tight_layout()
plt.show()

# Strongly Connected Components
find airport groupings for each airline where all the airports have flights to and from
all the other airports in that group

In [ ]:
def find_scc_components(g, airline):
    # Create a subgraph containing only flights on the provided airline
    airline_relationships = g.edges[g.edges.airline == airline]
    airline_graph = GraphFrame(g.vertices, airline_relationships)
    # Calculate the Strongly Connected Components
    scc = airline_graph.stronglyConnectedComponents(maxIter=10)
    # Find the size of the biggest component and return that
    return (scc
        .groupBy("component")
        .agg(F.count("id").alias("size"))
        .sort("size", ascending=False)
        .take(1)[0]["size"])

In [ ]:
# Calculate the largest strongly connected component for each airline
airline_scc = [(airline, find_scc_components(g, airline))
               for airline in airlines.toPandas()["airline"].tolist()]
airline_scc_df = spark.createDataFrame(airline_scc, ['id', 'sccCount'])
# Join the SCC DataFrame with the airlines DataFrame so that we can show
# the number of flights an airline has alongside the number of
# airports reachable in its biggest component
airline_reach = (airline_scc_df
                 .join(full_name_airlines, full_name_airlines.code == airline_scc_df.id)
                 .select("code", "name", "flights", "sccCount")
                 .sort("sccCount", ascending=False))

In [ ]:
ax = (airline_reach.toPandas()
      .plot(kind='bar', x='name', y='sccCount', legend=None))
ax.xaxis.set_label_text("")
plt.tight_layout()
plt.show()

Connected Components with Apache Spark

In [ ]:
# Now let’s say most of the frequent flyer points we have are with Delta Airlines (DL).
# Can we find airports that form communities within the network for that particular     airline carrier?
airline_relationships = g.edges.filter("airline = 'DL'")
airline_graph = GraphFrame(g.vertices, airline_relationships)

In [ ]:
clusters = airline_graph.labelPropagation(maxIter=10)
(clusters
 .sort("label")
 .groupby("label")
 .agg(F.collect_list("id").alias("airports"),
      F.count("id").alias("count"))
 .sort("count", ascending=False)
 .show(truncate=70, n=10))

In [ ]:
# calculate airport degree
all_flights = g.degrees.withColumnRenamed("id", "aId")

In [ ]:
(clusters
 .filter("label=1606317768706")
 .join(all_flights, all_flights.aId == clusters.id)
 .sort("degree", ascending=False)
 .select("id", "name", "degree")
 .show(truncate=False))

# Appendix: Timing and Profling
%time: Time the execution of a single statement

%timeit: Time repeated execution of a single statement for more accuracy

%prun: Run code with the profiler

%lprun: Run code with the line-by-line profiler

%memit: Measure the memory use of a single statement

%mprun: Run code with the line-by-line memory profile